In [1]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
total_df=pd.read_csv("risk_data/Train_data_set.csv", encoding='utf-8-sig')

In [3]:
total_df.head()

,month,day,시군구index,철새수,최초발생거리,평균온도,일강수량,위험도
0,1,1,0,2262,0,-1.8,0.0,8.4
1,1,1,0,2262,10,-1.8,0.0,8.4
2,1,1,0,2262,20,-1.8,0.0,8.4
3,1,1,0,2262,30,-1.8,0.0,8.4
4,1,1,0,2262,40,-1.8,0.0,8.4


In [4]:
train_data=total_df.drop(['위험도'], axis=1)
target_data=total_df[['위험도']].astype('str')

In [5]:
x_train, x_test, y_train, y_test = train_test_split(train_data, target_data, test_size = 0.2, random_state = 156)

In [6]:
lgbm_wrapper = LGBMClassifier()

In [7]:
evals = [(x_test, y_test)]
lgbm_wrapper.fit(x_train, np.ravel(y_train), early_stopping_rounds=50, eval_metric='logloss', eval_set=evals, verbose=True)

C:\Users\user\.conda\envs\my_env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[1]	valid_0's multi_logloss: 2.00783
Training until validation scores don't improve for 50 rounds
[2]	valid_0's multi_logloss: 1.96269
[3]	valid_0's multi_logloss: 2.03468
[4]	valid_0's multi_logloss: 2.37121
[5]	valid_0's multi_logloss: 2.61531
[6]	valid_0's multi_logloss: 2.82635
[7]	valid_0's multi_logloss: 2.98623
[8]	valid_0's multi_logloss: 3.08569
[9]	valid_0's multi_logloss: 3.22992
[10]	valid_0's multi_logloss: 3.28553
[11]	valid_0's multi_logloss: 3.71345
[12]	valid_0's multi_logloss: 3.97804
[13]	valid_0's multi_logloss: 4.08862
[14]	valid_0's multi_logloss: 4.47559
[15]	valid_0's multi_logloss: 4.56681
[16]	valid_0's multi_logloss: 4.76155
[17]	valid_0's multi_logloss: 4.6412
[18]	valid_0's multi_logloss: 5.00382
[19]	valid_0's multi_logloss: 5.29192
[20]	valid_0's multi_logloss: 5.6317
[21]	valid_0's multi_logloss: 5.70286
[22]	valid_0's multi_logloss: 6.01653
[23]	valid_0's multi_logloss: 6.53708
[24]	valid_0's multi_logloss: 7.08908
[25]	valid_0's multi_logloss: 7.76228


LGBMClassifier()

In [8]:
preds = lgbm_wrapper.predict(x_test) 
pred_proba = lgbm_wrapper.predict_proba(x_test)[:,1]

In [9]:
# get_clf_eval 활용 

# XGB 모델 예측 성능 평가 

from sklearn.metrics import confusion_matrix, accuracy_score 
from sklearn.metrics import precision_score, recall_score 
from sklearn.metrics import f1_score, roc_auc_score

In [10]:
def get_clf_eval(y_test , pred): 
    confusion = confusion_matrix(y_test, pred) 
    accuracy = accuracy_score(y_test , pred) 
    precision = precision_score(y_test , pred, average='micro') 
    recall = recall_score(y_test , pred, average='micro') 
    f1 = f1_score(y_test,pred, average='micro') 
    #print(y_test.shape)
    #print(pred.shape)
    print('오차 행렬') 
    print(confusion) 
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\ F1: {3:.4f}'.format(accuracy, precision, recall, f1)) 
    
get_clf_eval(y_test, preds)

오차 행렬
[[54128     0     0 ...     0     0     0]
 [  634  5591     0 ...     0     0     0]
 [    0     0 11085 ...     0     0     0]
 ...
 [    2     0     0 ...   114     0     0]
 [    0     0     0 ...    20   251     0]
 [    0     0     0 ...    20     0   129]]
정확도: 0.7211, 정밀도: 0.7211, 재현율: 0.7211,\ F1: 0.7211


In [ ]:
# 모델 저장
import joblib
joblib.dump(lgbm_wrapper, 'model/lightgbm2.pkl')